In [ ]:
'''
Created By Josh Stokley to create graph of drift vs shear ratio of simulated vs experimental results of RC walls
The script provides-
variables:
 refFile ........................... Defined txt file with variables related to RC wall
 node .............................. total nodes along width of wall
 name .............................. unique ID of wall (usually authors name + wall name)
 filename .......................... pathway to folder for the specific wall, holds reference file, tcl file, and recorded data
 h ................................. height of wall
 area .............................. area of cross section of wall
 fc ................................ strength of the concrete in the wall
 drift ............................. array of drift % of the top row of nodes simulated
 shearRatio ........................ array of shear ratio of the bottom row of nodes simulated
 exDrift ........................... array of drift % of the top row of nodes experimental
 exshearRatio ...................... array of shear ratio of the bottom row of nodes experimental
 
methods:
 def plot(refFile) .................................................... overall function that takes in refFile
 def getReact(filename) ............................................... forces in the x direction at each node along the base are imported
 def getDisp(filename) ................................................ displacements in the y direction at each node along the top are imported
 importing experimental results ....................................... experimental displacement and force of wall are imported and converted to drift and shear ratio
 plotting script section .............................................. experimental vs simulated drift and shear ratio are plotted to compare results
'''


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import io
import math


def plot(refFile,parm):
    with open(refFile, 'rt') as h:
        data = [line.split() for line in h]
        node = int(data[0][0])
        name = data[7][0]
        filename = data[8][0]

    with open(filename + '/onset.txt', 'rt') as h:
        data = [line.split() for line in h]
        exOn = float(data[0][0])
    
    #returns the sum of a row of data
    def getReact(filename):
        with open(filename, 'r') as h:
            data = [line.split() for line in h]
            x = []
            i = 0
            loc = 0
            maxStrength = 0
            for line in range(len(data)):
                if len(data[i])<node or data[i][-1] == '-':
                    break
                else:
                    for ii in range(0,node):
                        loc += float(data[i][ii])


                    loc = -loc
                    x.append(loc)
                    loc = 0
                    i += 1

        return x


    def getDisp(filename):
        with open(filename, 'r') as h:
            data = [line.split() for line in h]
            x = []
            i = 0
            loc = 0
            for line in range(len(data)):
                if len(data[i])<node or data[i][-1] == '-':
                    break

                else:
                    for ii in range(0,node):
                            loc += float(data[i][ii])

                    loc = loc/node
                    x.append(loc)
                    loc = 0
                    i += 1
            return x


    #Simulated response
    disp = getDisp(filename + 'topDispxcyc.txt')
    force = getReact(filename + 'baseReactxcyc.txt')

    #disp = getDisp('tcl_files/' + filename + '/topDispxcyc.txt')
    #force = getReact('tcl_files/' + filename + '/baseReactxcyc.txt')
    if len(disp) < len(force):
        force = force[:len(disp)]
    elif len(disp) > len(force):
        disp = disp[:len(force)]


    loaded = io.loadmat('WallData.mat')
    data = loaded['WallData']
    loc = data[0,0]["UniqueID"]
    locD = data[0,0]["ExperimentalData"][0,0][name][0,0]['disp']
    locF = data[0,0]["ExperimentalData"][0,0][name][0,0]['force']
    
    exDisp = []
    exForce = []
    for ii in range(len(locD)):
        exDisp.append(locD[ii][0])
    for ii in range(len(locF)):
        exForce.append(locF[ii][0])  
        
    #print(max(exForce))
    for ii in range(len(loc)):
        if name == loc[ii][0][0]:
            index = ii
            break
    
    h1 = float(data[0,0]["Geometry"][0,0]['h_measured'][index,0])
    h2 = float(data[0,0]["Geometry"][0,0]['h_v'][index,0])
    
    area = float(data[0,0]["Geometry"][0,0]['Area'][index,0])
    fc = float(data[0,0]["Material"][0,0]['fc'][index,0])
    if (h2<h1):
            h = h2
       
    else:
            h = h2
    h = h/12
    with open('mnmb.txt', 'r') as k:
        data = [line.split() for line in k]
        for ii in data:
            if ii[0] in name and ii[1] in name:
                mn = float(ii[4])
    #print(mn)
    exmnmb = []    
    for ii in exForce:
        exmnmb.append(ii*h/mn)
    mnmb = []    
    for ii in force:
        mnmb.append(ii*h/mn)
        
    drift = []
    shearRatio = []
    for ii in disp:
        drift.append(ii/(h*12)*100)
    for ii in force:
        shearRatio.append(ii*1000/(area*fc**(1/2)))
    
    
    exDrift = []
    exshearRatio = []
    for ii in exDisp:
        exDrift.append(ii/(h*12)*100)
    for ii in exForce:
        exshearRatio.append(ii*1000/(area*fc**(1/2))) 
    fig, ax = plt.subplots(figsize=(3.54331, 2.5),linewidth=1) #edgecolor="#04253a")
    

    ax.plot(exDrift,exmnmb, label = 'Experiment ', color = 'blue')
    ax.plot(drift,mnmb, label = 'Simulation',color = 'red')
    ax.grid(True)
    ax.set_ylabel("$M_{b}$/$M_{n}$", fontsize=15)
    ax.set_xlabel('Drift %', fontsize=15)
    ax.set_title(name, fontsize=15)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.legend(fontsize=15)
    #fig.show(dpi = 300)
    #fig.tight_layout()
    fig.savefig(filename+'DispLoad.png')#dpi = 1000)
    #plt.close(fig)


In [ ]:
files = [
    'tcl_files//tcl_files_23030421//SeguraWP7_230304211618//230304211618referenceFile.txt',
     'tcl_files//tcl_files_23030420//LowesPW4_230304205135//230304205135referenceFile.txt',
       'tcl_files//tcl_files_23022600//OhWR20_230226004714//230226004714referenceFile.txt',
 'tcl_files//tcl_files_23030120//ZhangWangSW8_230301204146//230301204146referenceFile.txt'
]

for ii in files:
    plot(ii,1)